In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os

In [10]:
import torch
from models.capsnet import CapsNet
from utils.datasets import GraphemesDataset
from torch.utils.data import DataLoader
from scripts.test_capsnet import train, test
from configs.capsnet_config import CapsNetConfig
import torchvision.transforms as transforms

torch.manual_seed(1)

USE_CUDA = True if torch.cuda.is_available() else False
BATCH_SIZE = 100
N_EPOCHS = 30

img_transform = transforms.Compose([
    transforms.ToTensor()
])

letter = 'aleph'

root_dir = "dataset_skia"
train_dataset = GraphemesDataset(root_dir, train=True, by_letter=letter, transform=img_transform)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

test_dataset = GraphemesDataset(root_dir, train=False, by_letter=letter, transform=img_transform)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

n_classes = len(train_dataset.classes)

In [12]:
config = Config(dataset='mnist')
config.dc_num_capsules = n_classes

capsule_net = CapsNet(config)
capsule_net = torch.nn.DataParallel(capsule_net)
if USE_CUDA:
    capsule_net = capsule_net.cuda()
capsule_net = capsule_net.module

optimizer = torch.optim.Adam(capsule_net.parameters())

for e in range(1, N_EPOCHS + 1):
    train(capsule_net, optimizer, train_loader, e, n_classes=n_classes)
    test(capsule_net, test_loader, e, n_classes=n_classes)

  0%|          | 0/53 [00:01<?, ?it/s]


RuntimeError: The size of tensor a (13) must match the size of tensor b (10) at non-singleton dimension 1